# SKA SDP AA0.5LOW Emulated Receive Workflow

Instructions to testing the sdp receive workflow using emulated AA0.5LOW data with tango and sdp interfaces against either the persistent sdp or a custom sdp deployment. This notebook can be executed remotely using binderhub via the following link:

https://sdhp.stfc.skao.int/binderhub/v2/gl/ska-telescope%2Fsdp%2Fska-sdp-notebooks/HEAD

## Tango Device Proxy Interface Intro

The Tango device proxy interface provides interaction to a subarray and it's associated execution block in the form of a state machine. When the device is On, this interface provides a single observable state object:

#### ObsState

* EMPTY
* IDLE
* READY
* SCANNING
* FAULT

Communication to the Tango device is performed via the use of commands and accessors whereby all data is conformant to the sdp schemas available here:

https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp.html


### Interface Schemas

| Command    | Current State | Next State | Input  | Output |
| ---------- | ------------- | ---------- | ------ | ------ |
| AssignRes  |     EMPTY     |   IDLE     | https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-assignres.html | None
| Configure  |     IDLE      |   READY    | https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-configure.html | None
| Scan       |     READY     |  SCANNING  | https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-scan.html | None
| RecvAddrs  |    SCANNING   |  SCANNING  | None | https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-recvaddrs.html
| EndScan    |    SCANNING   |   READY    | None | None
| End        |     READY     |   IDLE     | None | None
| ReleaseRes |     IDLE      |   EMPTY    | None | None

Note: The next state transition is not instantaneous and should be waited for before executing another command.

### Execution Block

The tango device monitors exactly 1 execution block that is defined by the assign resources command.

### Processing Block

Individual processes running in an execution block when the tango device has resources assigned and is not in the EMPTY state.

# Install Kubernetes (Development)

In [1]:
# Debug using python kubernetes
!pip install kubernetes

# Alternatively debug using kubectl commands
!curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
!sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl
!kubectl --help

# In addition to this step, you must provide a kubeconfig file which is by default located at:
# $HOME/.kube/config
# or KUBECONFIG environment variable 
# or can be passed to load_kube_config()

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artefact.skao.int/repository/pypi-internal/simple, https://pypi.org/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 KB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 KB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 27.9 MB/s eta 0:00:00
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   138  100   138    0     0   1232      0 --:--:-- --:--:-- --:--:--  1232
100 42.9M  100 42.9M    0     0  99.3M      0 --:--:-- --:--:-- --:--:-- 99.3M
kubectl controls the Kubernetes cluster manager.

 Find more information at: https://kubernetes.io/docs/reference/kubectl/

Basic Commands (Beginner):
  create          Create a res

# Install Helm (Development)

In [2]:
!sudo apt install -y gpg
!curl https://baltocdn.com/helm/signing.asc | gpg --dearmor | sudo tee /usr/share/keyrings/helm.gpg > /dev/null
!sudo apt-get install apt-transport-https --yes
!echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/helm.gpg] https://baltocdn.com/helm/stable/debian/ all main" | sudo tee /etc/apt/sources.list.d/helm-stable-debian.list
!sudo apt-get update
!sudo apt-get install helm
!helm --help

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  dirmngr gnupg gnupg-l10n gnupg-utils gpg-agent gpg-wks-client gpg-wks-server
  gpgconf gpgsm libassuan0 libksba8 libnpth0 pinentry-curses
Suggested packages:
  dbus-user-session libpam-systemd pinentry-gnome3 tor parcimonie xloadimage
  scdaemon pinentry-doc
The following NEW packages will be installed:
  dirmngr gnupg gnupg-l10n gnupg-utils gpg gpg-agent gpg-wks-client
  gpg-wks-server gpgconf gpgsm libassuan0 libksba8 libnpth0 pinentry-curses
0 upgraded, 14 newly installed, 0 to remove and 0 not upgraded.
Need to get 7089 kB of archives.
After this operation, 14.9 MB of additional disk space will be used.
Get:1 http://deb.debian.org/debian buster/main amd64 libassuan0 amd64 2.5.2-1 [49.4 kB]
Get:2 http://deb.debian.org/debian buster/main amd64 gpgconf amd64 2.2.12-1+deb10u1 [510 kB]
Get:3 http://deb.debian.org/debian buster/main amd64 

# Connect to SDP

The SDP is a helm chart in https://gitlab.com/ska-telescope/sdp/ska-sdp-integration deployed to a namespace on the kubernetes cluster. `dp-shared` is the name of the persistent SDP namespace. For testing the following script can alternatively deploy to a custom personal or team namespace.

In [ ]:
# Restart Kernel to refresh environment variables
import os
os._exit(00)

In [1]:
from tango import DeviceProxy, EventType
import ska_sdp_config
import os
import json
import random
from datetime import date
import logging
import ska_ser_logging

debug = False  # kubernetes client requires a kubeconfig and is only available for development 
shared = True
deploy_sdp = False  # if not shared, also deploy sdp
dev_sdp = False  # use the latest unreleased sdp

if debug:
    import kubernetes
    KUBECONFIG = "/app/config" # or "$HOME/.kube/config"
    k8s_client = kubernetes.client.api_client.ApiClient(configuration=kubernetes.config.load_kube_config(KUBECONFIG))
    k8s_core = kubernetes.client.CoreV1Api()
    k8s_batch = kubernetes.client.BatchV1Api()

if shared:
    # Make sure you connect to the correct Configuration Database
    KUBE_NAMESPACE = "dp-shared"
    KUBE_PROC_NAMESPACE = f"{KUBE_NAMESPACE}-p"
else:
    KUBE_NAMESPACE = "dp-yanda"  # add the namespace you want to connect to here
    KUBE_PROC_NAMESPACE = f"{KUBE_NAMESPACE}-p"
    # deploy the sdp
    if debug and deploy_sdp:
        if not dev_sdp:
            !helm repo add ska https://artefact.skao.int/repository/helm-internal
            !KUBECONFIG={KUBECONFIG} helm uninstall persistent-sdp --namespace {KUBE_NAMESPACE} --wait
            !KUBECONFIG={KUBECONFIG} helm upgrade --install persistent-sdp ska/ska-sdp --namespace {KUBE_NAMESPACE} --set helmdeploy.namespace={KUBE_PROC_NAMESPACE} --wait
        else:    
            # Alternative: sdp install from local repo
            !apt install git
            !git clone https://gitlab.com/ska-telescope/sdp/ska-sdp-integration.git --init --recursive
            !KUBECONFIG={KUBECONFIG} make uninstall-sdp -C ./ska-sdp-integration KUBE_NAMESPACE={KUBE_NAMESPACE} KUBE_NAMESPACE_SDP={KUBE_PROC_NAMESPACE} --wait
            !KUBECONFIG={KUBECONFIG}" make install-sdp -C ./ska-sdp-integration KUBE_NAMESPACE={KUBE_NAMESPACE} KUBE_NAMESPACE_SDP={KUBE_PROC_NAMESPACE}
    
# set the name of the databaseds service
DATABASEDS_NAME = "databaseds-tango-base"

# finally set the predetermined host name
os.environ["SDP_CONFIG_HOST"] = f"ska-sdp-etcd-client.{KUBE_NAMESPACE}"
os.environ["TANGO_HOST"] = f"{DATABASEDS_NAME}.{KUBE_NAMESPACE}.svc.cluster.local:10000"

In [2]:
# tango device
d = DeviceProxy('test-sdp/subarray/01')
d.set_logging_level(5)

# sdp config
config = ska_sdp_config.Config()

# ska logging
logger = logging.getLogger(__name__)
ska_ser_logging.configure_logging(level=logging.INFO)

In [3]:
# Check SDP deployment
if debug:
    !KUBECONFIG={KUBECONFIG} helm list -n {KUBE_NAMESPACE}
    !KUBECONFIG={KUBECONFIG} kubectl get pods -n {KUBE_NAMESPACE}
    !KUBECONFIG={KUBECONFIG} kubectl get pods -n {KUBE_PROC_NAMESPACE}

NAME          	NAMESPACE      	REVISION	UPDATED                                	STATUS  	CHART         	APP VERSION
persistent-sdp	dp-yanda-callan	1       	2022-09-27 03:43:06.127557129 +0000 UTC	deployed	ska-sdp-0.11.2	0.11.2     
NAME                              READY   STATUS      RESTARTS   AGE
databaseds-tango-base-0           1/1     Running     0          164m
ska-sdp-console-0                 1/1     Running     0          164m
ska-sdp-etcd-0                    1/1     Running     0          164m
ska-sdp-helmdeploy-0              1/1     Running     0          164m
ska-sdp-lmc-config--1-9gb5z       0/1     Completed   0          164m
ska-sdp-lmc-controller-0          1/1     Running     0          164m
ska-sdp-lmc-subarray-01-0         1/1     Running     0          164m
ska-sdp-opinterface-0             1/1     Running     0          164m
ska-sdp-proccontrol-0             1/1     Running     0          164m
ska-sdp-scripts-config--1-skvxk   0/1     Completed   0          164m

In [12]:
# Utilities
from tango import DevState
import pytest
import time
import datetime
from typing import Optional

TIMEOUT = 60.0  # seconds
INTERVAL = 0.5  # seconds

def wait_for_predicate(
    predicate, description, timeout=TIMEOUT, interval=INTERVAL
):
    """
    Wait for predicate to be true.

    :param predicate: callable to test
    :param description: description to use if test fails
    :param timeout: timeout in seconds
    :param interval: interval between tests of the predicate in seconds

    """
    start = time.time()
    while True:
        if predicate():
            break
        if time.time() >= start + timeout:
            pytest.fail(f"{description} not achieved after {timeout} seconds")
        time.sleep(interval)


def wait_for_state(device, state, timeout=TIMEOUT):
    """
    Wait for device state to have the expected value.

    :param device: device client
    :param state: the expected state
    :param timeout: timeout in seconds

    """

    def predicate():
        return device.state() == state

    description = f"Device state {state.name}"
    logger.info(f"Waiting for device state {state.name}...")
    wait_for_predicate(predicate, description, timeout=timeout)


def wait_for_obs_state(device, obs_state, timeout=TIMEOUT):
    """
    Wait for obsState to have the expected value.

    :param device: device proxy
    :param obs_state: the expected value
    :param timeout: timeout in seconds
    """

    def predicate():
        return device.obsState == obs_state

    description = f"obsState {obs_state.name}"
    logger.info(f"Waiting for device obs_state {obs_state.name}...")
    wait_for_predicate(predicate, description, timeout=timeout)

def tango_safe_release(device):
    """
    Safely releases tango device to EMPTY obsState
    """
    if device.obsState == device.obsState.SCANNING:
        logger.info(">> End Scan")
        device.EndScan()
        wait_for_obs_state(device, device.obsState.READY)

    if device.obsState == device.obsState.READY:
        logger.info(">> End")
        device.End()
        wait_for_obs_state(device, device.obsState.IDLE)

    if device.obsState == device.obsState.IDLE:
        logger.info(">> Releasing Resources")
        device.ReleaseResources()        
        wait_for_obs_state(device, device.obsState.EMPTY)
        
    if device.obsState == device.obsState.FAULT:
        device.Restart()
        wait_for_obs_state(device, device.obsState.EMPTY)
        
    assert device.obsState == device.obsState.EMPTY
    logger.info("Tango Device is EMPTY")
    
def tango_safe_off(device):
    """
    Safely turns tango device to OFF state
    """
    tango_safe_release(device)
    if device.state() == DevState.ON:
        logger.info(">> Device OFF")
        device.Off()
        wait_for_state(device, DevState.OFF)

    assert device.state() == DevState.OFF
    logger.info("Tango Device is OFF")

# Debug
def pod_status(namespace: str, name: str) -> Optional[str]:
    pods = k8s_core.list_namespaced_pod(namespace=namespace)
    for i in pods.items:
        if name == i.metadata.name:
            return i.status.phase
    return None

def print_pods(namespace: str):
    pods = k8s_core.list_namespaced_pod(namespace=namespace)
    print(f"pod_ip\t\tnamespace\tname\t\t\t\t\t\tphase")
    for i in pods.items:
        print(f"{str(i.status.pod_ip):10}\t{i.metadata.namespace}\t{i.metadata.name:20}\t{i.status.phase}")

def print_pod_events(namespace: str, name: str):
    field_selector = f"involvedObject.name={name}"
    stream = kubernetes.watch.Watch().stream(k8s_core.list_namespaced_event, namespace=namespace, field_selector=field_selector, timeout_seconds=1)
    print(f"Events for {receive_pod_name}:")
    print("Type\tReason\t\t\tAge\t\tFrom\t\t\tMessage")
    for event in stream:
        print(
            f"{event['object'].type}\t"
            f"{event['object'].reason}\t"
            f"{event['object'].event_time}\t"
            f"{event['object'].reporting_component}\t"
            f"{event['object'].message}"
        )

def print_job_logs(namespace: str, name: str, container: str):
    label_selector = f"job-name={name}"
    pods = sorted(
        k8s_core.list_namespaced_pod(namespace=namespace, label_selector=label_selector).items,
        key=lambda p: p.metadata.creation_timestamp
    )
    pod_name = pods[-1].metadata.name
    print("Logs for:", pod_name)
    print(k8s_core.read_namespaced_pod_log(namespace=KUBE_PROC_NAMESPACE, name=pod_name, container=container))

In [13]:
tango_safe_off(d)

1|2022-09-27T08:05:02.885Z|INFO|MainThread|tango_safe_release|4124581261.py#90||Tango Device is EMPTY
1|2022-09-27T08:05:02.886Z|INFO|MainThread|tango_safe_off|4124581261.py#103||Tango Device is OFF


# Configure Execution Block

To create an execution block, parameters need to be passed to the workflow script and helm charts in ska-sdp-scripts to ensure correct reception is configured.

In [6]:
# list of available workflows
!ska-sdp list script

Keys with prefix /script: 
/script/batch:test-batch:0.3.0
/script/batch:test-daliuge:0.3.0
/script/batch:test-dask:0.3.0
/script/realtime:pss-receive:0.3.0
/script/realtime:test-realtime:0.3.0
/script/realtime:test-realtime:0.4.0
/script/realtime:test-receive-addresses:0.4.0
/script/realtime:test-receive-addresses:0.5.0
/script/realtime:vis-receive:0.5.0
/script/realtime:vis-receive:0.5.1
/script/realtime:vis-receive:0.6.0


In [14]:
# Config Globals
total_channels      = 13824
total_streams       = 4
rate                = 10416667  # bits per second
channels_per_stream = total_channels // total_streams

total_timesteps     = 6
num_repeats         = 3


def create_receive_parameters(eb_id: str) -> dict:
    max_payloads        = total_timesteps * total_streams * num_repeats
    max_payload_misses  = 30  # payload timeout in seconds
    max_ms              = 1  # -1 to continuously run
    
    return {
        "image": "artefact.skao.int/ska-sdp-realtime-receive-modules",
        "version": "3.3.0",
        "pvc": {
            "name": "shared"
        },
        "reception": {
            "num_channels": total_channels,
            "channels_per_stream": channels_per_stream,
            "execution_block_id": eb_id, # alternatives are schedblock filename or datamodel filename
            "layout": "http://127.0.0.1:80/model/default/ska1_low/layout",
            "sdp_config_backend": "etcd3",
            "sdp_config_host": os.environ["SDP_CONFIG_HOST"],
            "sdp_config_port": 2379,
            "continuous_mode": True,
            "transport_protocol": "udp"
        },
        "plasmaEnabled": True,
        "plasma_parameters": {
            "extraContainers": [
                {
                    "name": "tmlite-server",
                    "image": "artefact.skao.int/ska-sdp-tmlite-server:0.3.0"
                }
            ]
        }
    }


def create_resources_config():
    generator = "notebook"
    today = date.today().strftime("%Y%m%d")
    number = random.randint(0, 99998)

    EXECUTION_BLOCK_ID = f"eb-{generator}-{today}-{number:05d}"
    PROCESSING_BLOCK_ID_REALTIME_SENDER = f"pb-{generator}-{today}-{number:05d}"
    PROCESSING_BLOCK_ID_REALTIME_RECEIVER = f"pb-{generator}-{today}-{number+1:05d}"
    PROCESSING_BLOCK_ID_BATCH = f"pb-{generator}-{today}-{number+2:05d}"

    return {
      "interface": "https://schema.skao.int/ska-sdp-assignres/0.3",
      "eb_id":f"{EXECUTION_BLOCK_ID}",
      "max_length": 21600.0,
      "resources": {  # also required in 0.3
        "csp_links": [1, 1],
        "receive_nodes": 1,
        "receptors":["C1", "C2", "C3", "C4"]
      },
      "scan_types": [
        {
          "scan_type_id": "science",
          "beams": { "0": { "channels_id": "vis_channels" } },
          "coordinate_system": "ICRS", "ra": "02:42:40.771", "dec": "-00:00:47.84",
          "channels": [
            { "count": 10, "start": 0, "stride": 2, "freq_min": 0.35e9, "freq_max": 0.368e9, "link_map": [[0,0], [200,1], [744,2], [944,3]] },
          ]
        },
        {
          "scan_type_id": "calibration",
          "beams": { "0": { "channels_id": "vis_channels" } },
          "coordinate_system": "ICRS", "ra": "12:29:06.699", "dec": "02:03:08.598",
          "channels": [
            { "count": 5, "start": 0, "stride": 2, "freq_min": 0.35e9, "freq_max": 0.368e9, "link_map": [[0,0], [200,1], [744,2], [944,3]] },
          ]
        }
      ],
      "channels": [
        {
            "channels_id": "vis_channels",
            "spectral_windows": [
                {
                    "spectral_window_id": "all_channels",
                    "count": 4, "start": 0, "stride": 2,
                    "freq_min": 0.35e9, "freq_max": 0.368e9,
                    "link_map": [[0,0], [200,1], [744,2], [944,3]]
                }
            ]
        }
      ],
      "processing_blocks": [
          {
            "pb_id": f"{PROCESSING_BLOCK_ID_REALTIME_RECEIVER}",
            "workflow": {"kind": "realtime", "name": "vis-receive", "version": "0.5.1"},
            "parameters": create_receive_parameters(EXECUTION_BLOCK_ID)
          }
      ]
    }

# Install and run CBF-Emulator with data from Jupyter

The simplest approach for synchronizing scans between sender and receiver is to run the cbf-emulator packetizer from the runtime controlling the tango device. 

In [15]:
!sudo apt install -y wget iputils-ping
!pip install ska-sdp-cbf-emulator
!wget -qnc https://gitlab.com/ska-telescope/sdp/ska-sdp-realtime-receive-core/-/raw/main/data/AA05LOW.ms.tar.gz
!tar -xzf AA05LOW.ms.tar.gz

import cbf_sdp.packetiser
from realtime.receive.core.config import create_config_parser

async def cbf_scan(target_host: str, target_port: str, scan_id: int):
    # Same as:
    #!emu-send\
    #-o transmission.method=spead2_transmitters\
    #-o transmission.channels_per_stream={channels_per_stream}\
    #-o transmission.rate={rate}\
    #-o transmission.target_host={host}\
    #-o transmission.target_start_port={port}\
    #-o reader.num_repeats={num_repeats}\
    #AA05LOW.ms

    sender_args = create_config_parser()
    sender_args['reader'] = {
        'scan_ids': [scan_id],
        'num_repeats': num_repeats
    }
    sender_args['transmission'] = {
        'method': 'spead2_transmitters',
        'channels_per_stream': channels_per_stream,
        'rate': rate,
        'target_host': target_host,
        'target_port': target_port
    }
    await cbf_sdp.packetiser.packetise(sender_args, "AA05LOW.ms")    


Reading package lists... Done
Building dependency tree       
Reading state information... Done
iputils-ping is already the newest version (3:20180629-2+deb10u2).
wget is already the newest version (1.20.1-1.1).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artefact.skao.int/repository/pypi-internal/simple, https://pypi.org/simple


# Send-Receive Emulated Scans

Using the cbf-emulator library and tango device, emulated scan reception can be performed by configuring the tango device into the SCANNING state and calling the cbf_scan function to send packets directly from notebook to receiver pod.

In [10]:
import sys

tango_safe_release(d)
try:
    if d.state() == DevState.OFF:
        print(">> Device ON")
        d.On()
        wait_for_state(d, DevState.ON)
        wait_for_obs_state(d, d.obsState.EMPTY)

    print(">> Assigning Resources")
    config_eb = create_resources_config()
    # https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-assignres.html
    d.AssignResources(json.dumps(config_eb))
    wait_for_obs_state(d, d.obsState.IDLE, timeout=120)
    
    # patch ebconfig for pre vis-receive:0.6.0
    if config_eb["interface"] == "https://schema.skao.int/ska-sdp-assignres/0.3":
        for txn in config.txn():
            eb = txn.get_execution_block(config_eb["eb_id"])
            eb["resources"] = config_eb["resources"]
            txn.update_execution_block(config_eb["eb_id"], eb)

    print(">> Get Receive Address")
    receiveAddresses = json.loads(d.receiveAddresses)
    host = receiveAddresses["science"]["host"][0][1]
    port = receiveAddresses["science"]["port"][0][1] # start_port
    print(host, port)
    # wait for receiver address
    !while true; do sleep 5; ping -c1 {host} > /dev/null && break; done
    print("Host found")
    
    for scan_type in ["science"]:
        print(">> Configure")
        # https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-configure.html
        d.Configure(json.dumps({"interface": "https://schema.skao.int/ska-sdp-configure/0.3", "scan_type": "science"}))
        wait_for_obs_state(d, d.obsState.READY)


        for scan_id in range(4):
            print(">> Scan")
            d.Scan(json.dumps({"interface": "https://schema.skao.int/ska-sdp-scan/0.3", "scan_id": scan_id}))
            wait_for_obs_state(d, d.obsState.SCANNING)

            time.sleep(10)
            await cbf_scan(host, port, scan_id)
            time.sleep(10)

            print(">> End Scan")
            d.EndScan()
            wait_for_obs_state(d, d.obsState.READY)

        print(">> End")
        d.End()
        wait_for_obs_state(d, d.obsState.IDLE)

    print(">> Releasing Resources")
    d.ReleaseResources()
    wait_for_obs_state(d, d.obsState.EMPTY)

    print(">> Device OFF")
    d.Off()
    wait_for_state(d, DevState.OFF)

except Exception as e:
    # In case of failure, safely restore the device back to its original off state
    tango_safe_release(d)
    raise e

1|2022-09-27T06:29:41.154Z|INFO|MainThread|tango_safe_release|3322442211.py#89||Tango Device is EMPTY
>> Assigning Resources
1|2022-09-27T06:29:41.214Z|INFO|MainThread|wait_for_obs_state|3322442211.py#62||Waiting for device obs_state IDLE...
>> Get Receive Address
proc-pb-notebook-20220927-54313-receive-0.receive.dp-yanda-callan-p.svc.cluster.local 41000
ping: proc-pb-notebook-20220927-54313-receive-0.receive.dp-yanda-callan-p.svc.cluster.local: Name or service not known
ping: proc-pb-notebook-20220927-54313-receive-0.receive.dp-yanda-callan-p.svc.cluster.local: Name or service not known
ping: proc-pb-notebook-20220927-54313-receive-0.receive.dp-yanda-callan-p.svc.cluster.local: Name or service not known
ping: proc-pb-notebook-20220927-54313-receive-0.receive.dp-yanda-callan-p.svc.cluster.local: Name or service not known
ping: proc-pb-notebook-20220927-54313-receive-0.receive.dp-yanda-callan-p.svc.cluster.local: Name or service not known
ping: proc-pb-notebook-20220927-54313-receive-0.

# Send-Receive Emulated Scans (ContextManager)

Cleanup commands can be ensured and encapsulated using a ContextManager interface and remove bloat.

In [18]:
# Encapsulate Scans

from typing import ContextManager

class SubarrayDevice(ContextManager):
    def __init__(self, device: DeviceProxy):
        self._device = device
    
    def __enter__(self):
        logger.info(">> Device ON")
        self._device.On()
        wait_for_state(self._device, DevState.ON)
        wait_for_obs_state(self._device, d.obsState.EMPTY)
        return self
    
    def __exit__(self, t, v, tb):
        logger.info(">> Device OFF")
        self._device.Off()
        wait_for_state(d, DevState.OFF)
    
    def assign_resources(self, config: dict, timeout=60):
        # TODO: pass self
        return ScanDeviceResources(self._device, config, timeout)


class SubarrayDeviceResources(ContextManager):
    def __init__(self, device: DeviceProxy, config: dict, timeout: int):
        self._device = device
        # https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-assignres.html
        self._config = config
        self._timeout = timeout
    
    def __enter__(self):
        logger.info(">> Assigning Resources")
        self._device.AssignResources(json.dumps(self._config))
        wait_for_obs_state(self._device, d.obsState.IDLE, timeout=self._timeout)
        # patch ebconfig
        for txn in ska_sdp_config.Config().txn():
            eb = txn.get_execution_block(self._config["eb_id"])
            eb["resources"] = self._config["resources"]
            txn.update_execution_block(self._config["eb_id"], eb)
        return self
    
    def __exit__(self, t, v, tb):
        logger.info(">> Releasing Resources")
        d.ReleaseResources()        
        wait_for_obs_state(d, d.obsState.EMPTY)

    def configure(self, scan_type: str):
        return ScanDeviceConfiguration(self._device, scan_type)


class SubarrayDeviceConfiguration(ContextManager):
    def __init__(self, device: DeviceProxy, scan_type: str):
        self._device = device
        self._scan_type = scan_type
        
    def __enter__(self):
        logger.info(">> Configure")
        # https://developer.skao.int/projects/ska-telmodel/en/latest/schemas/ska-sdp-configure.html
        self._device.Configure(json.dumps(
            {"interface": "https://schema.skao.int/ska-sdp-configure/0.3", "scan_type": self._scan_type}
        ))
        wait_for_obs_state(self._device, self._device.obsState.READY)
        
        receiveAddresses = json.loads(self._device.receiveAddresses)
        self.host = receiveAddresses[self._scan_type]["host"][0][1]
        self.port = receiveAddresses[self._scan_type]["port"][0][1]
        !while true; do sleep 5; ping -c1 {self.host} > /dev/null && break; done
        logger.info("host found")
        return self
    
    def __exit__(self, t, v, tb):
        logger.info(">> End")
        self._device.End()
        wait_for_obs_state(d, d.obsState.IDLE)
    
    def scan(self, scan_id: int):
        return ScanDeviceScan(self._device, scan_id)


class SubarrayDeviceScan(ContextManager):
    def __init__(self, device: DeviceProxy, scan_id: int):
        self._device = device
        self._scan_id = scan_id
        
    def __enter__(self):
        logger.info(">> Scan")
        self._device.Scan(json.dumps(
            {"interface": "https://schema.skao.int/ska-sdp-scan/0.3", "scan_id": self._scan_id}
        ))
        wait_for_obs_state(self._device, self._device.obsState.SCANNING)
        return self
    
    def __exit__(self, t, v, tb):
        logger.info(">> End Scan")
        self._device.EndScan()
        wait_for_obs_state(self._device, self._device.obsState.READY)
    

In [19]:
import ska_ser_logging
import logging
ska_ser_logging.configure_logging(level=logging.INFO)

d = DeviceProxy('test-sdp/subarray/01')
tango_safe_release(d)
with SubarrayDevice(d) as device, device.assign_resources(create_resources_config(), timeout=120) as scan_resources:
    # Configure for each scan_type
    for scan_type in ["science"]:
        with resources.configure(scan_type) as scan_configuration:
            # Perform 3 scans
            for scan_id in range(3):
                with scan_configuration.scan(scan_id) as scan:
                    # TODO: scan and endscan commands change state too quickly  
                    time.sleep(10)
                    await cbf_scan(scan_configuration.host, scan_configuration.port, scan_id)
                    time.sleep(10)

1|2022-09-27T08:05:53.404Z|INFO|MainThread|tango_safe_release|4124581261.py#90||Tango Device is EMPTY
1|2022-09-27T08:05:53.404Z|INFO|MainThread|__enter__|1064687953.py#10||>> Device ON
1|2022-09-27T08:05:53.431Z|INFO|MainThread|wait_for_state|4124581261.py#46||Waiting for device state ON...
1|2022-09-27T08:05:53.438Z|INFO|MainThread|wait_for_obs_state|4124581261.py#63||Waiting for device obs_state EMPTY...
1|2022-09-27T08:05:53.439Z|INFO|MainThread|__enter__|3772511796.py#34||>> Assigning Resources
1|2022-09-27T08:05:53.496Z|INFO|MainThread|wait_for_obs_state|4124581261.py#63||Waiting for device obs_state IDLE...
1|2022-09-27T08:05:57.055Z|INFO|MainThread|__enter__|3772511796.py#59||>> Configure
1|2022-09-27T08:05:57.251Z|INFO|MainThread|wait_for_obs_state|4124581261.py#63||Waiting for device obs_state READY...
ping: proc-pb-notebook-20220927-41243-receive-0.receive.dp-yanda-callan-p.svc.cluster.local: Name or service not known
ping: proc-pb-notebook-20220927-41243-receive-0.receive.d